## Introduction
    indian government announced that heart diseases is the top one killer over the world. There are seventeen million people died from it every year, especially heart disease. Prevention is better than cure. If we can evaluate the risk of every patient who probably has heart disease, that is, not only patients but also everyone can do something earlier to keep illness away.

This  heart dataset is a real data including important features of patients. This time we will build the predictable model by XGBoost library. Before predict the test dataset, use concept of crossvalid to find the optimised parameters. after that, the model can calculate the weight of each features, so we can easily understand which feature is more influent than others.

pulmnory veins is a common technique to figure out the accuracy of the model. From the standpoint of medicine, the recall rate is more important than precision rate because no one want to be misdiagnosed if the one actually have heart disease. So we will check the recall performance. After that, roc curve can help us evaluate the model, and then we'll explore the features if the model is good enough.

 this chest pain is the more dangerous pain for male or feamle at a age of 55 for sme people it may attact very eralier too . so be habitutaul to good this , dont addict to the unwanted this as smoking , drinking alchol and many more.

 cvvc

## Exploratory Analysis
There are thirteen features and one target as below:
* age: The person's age in years
* sex: The person's sex (1 = male, 0 = female)
* cp: The chest pain experienced (Value 1: typical angina, Value 2: atypical angina, Value 3: non-anginal pain, Value 4: asymptomatic)
* trestbps: The person's resting blood pressure (mm Hg on admission to the hospital)
* chol: The person's cholesterol measurement in mg/dl
* fbs: The person's fasting blood sugar (> 120 mg/dl, 1 = true; 0 = false)
* restecg: Resting electrocardiographic measurement (0 = normal, 1 = having ST-T wave abnormality, 2 = showing probable or definite left ventricular hypertrophy by Estes' criteria)
* thalach: The person's maximum heart rate achieved
* exang: Exercise induced angina (1 = yes; 0 = no)
* oldpeak: ST depression induced by exercise relative to rest 
* slope: the slope of the peak exercise ST segment (Value 1: upsloping, Value 2: flat, Value 3: downsloping)
* ca: The number of major vessels (0-3)
* thal: A blood disorder called thalassemia (3 = normal; 6 = fixed defect; 7 = reversable defect)
    
* target: Heart disease (0 = no, 1 = yes)



In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import os
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import jaccard_similarity_score
from sklearn import metrics
from sklearn import preprocessing
import itertools
from sklearn.model_selection import validation_curve
from xgboost import XGBClassifier
import xgboost as xgb
from xgboost import plot_importance
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.inspection.partial_dependence import partial_dependence, plot_partial_dependence
from sklearn.ensemble import GradientBoostingClassifier
import eli5
from eli5.sklearn import PermutationImportance
import shap
from mpl_toolkits.mplot3d import Axes3D

There is 1 csv file in the current version of the dataset:


In [ ]:
print(os.listdir('../input'))

In [ ]:
filename = "../input/heart.csv"
df = pd.read_csv(filename)
df.head()

Let's see how percentage of the people who has heart disease.

In [ ]:
df_target = df.groupby("target").size()
df_target

In [ ]:
plt.pie(df_target.values, labels = ["target 0", "target 1"], autopct='%1.1f%%', radius = 1.5, textprops = {"fontsize" : 16}) 
plt.show()

Over 50% people who was diagnosed heart disease

In [ ]:
df_sex = df.groupby(["sex","target"]).size()
df_sex

In [ ]:
plt.pie(df_sex.values, labels = ["sex_0,target_0", "sex_0,target_1", "sex_1,target_0", "sex_1,target_1"],autopct='%1.1f%%',radius = 1.5, textprops = {"fontsize" : 16})
plt.show()

The ratio of male has heart disease is 30.7%, a little bit higher than female.

Let's see how distribution of age is.

In [ ]:
plt.hist([df[df.target==0].age, df[df.target==1].age], bins = 20, alpha = 0.5, label = ["no_heart_disease","with heart disease"])
plt.xlabel("age")
plt.ylabel("percentage")
plt.legend()
plt.show()

The ratio get higher over the age of forty. That is, people who is over forty is under high risk of heart disease.

In [ ]:
plt.hist([df[df.target==0].chol, df[df.target==1].chol], bins = 20, alpha = 0.5, label = ["no_heart_disease","with heart disease"])
plt.xlabel("chol")
plt.ylabel("percentage")
plt.legend()
plt.show()

Also, amounts of people having heart disease are over 200mg/dl of chol. According to the research, the normal value of chol should be lower than 200mg/dl.

How about the resting blood pressure?

In [ ]:
plt.hist([df[df.target==0].trestbps, df[df.target==1].trestbps], bins = 20, alpha = 0.5, label = ["no_heart_disease","with heart disease"])
plt.xlabel("trestbps")
plt.ylabel("percentage")
plt.legend()
plt.show()

The ideal blood pressure should be lower than 120 mmHg. Whether the patients have heart disease or not , over 50% patients have higher blood pressure.

Maximum heart rate is negatively related to the age. Let's see the distribution.

In [ ]:
plt.hist([df[df.target==0].thalach, df[df.target==1].thalach], bins = 20, alpha = 0.5, label = ["no_heart_disease","with heart disease"])
plt.xlabel("thalach")
plt.ylabel("percentage")
plt.legend()
plt.show()

Interesting outcome. It seems that the medium of patient with heart disease is higher. 

Before modeling the dataset, let's find out if there are outliers inside the dataset.
We check the contineous data by boxplot. 

In [ ]:
df_1 = df[["age", "trestbps", "chol", "thalach", "oldpeak"]]
df_1.describe()

In [ ]:
for item in df_1.columns:
    plt.subplot(3,2,list(df_1.columns).index(item)+1)
    plt.boxplot(df_1[item], patch_artist=True, labels = [item])
    plt.ylabel("value")
plt.tight_layout()
plt.show()

There are outliers in each features except age. We define the outliers that are not in the range of 3 sigma. Then remove the outliers.

In [ ]:
infor = df.describe()

df2 = df[df.trestbps < infor.loc["mean", "trestbps"] + 3 * infor.loc["std", "trestbps"]]
df3 = df2[df.chol < infor.loc["mean", "chol"] + 3 * infor.loc["std", "chol"]]
df4 = df3[df.thalach > infor.loc["mean", "thalach"] - 3 * infor.loc["std", "thalach"]]
df_new = df4[df.oldpeak < infor.loc["mean", "oldpeak"] + 3 * infor.loc["std", "oldpeak"]]
df_new.head()

Replace the name of category data and seperate them into different columns.

* OBJECTIVES: THE MAIN OBJECTIVE OF THIS RESEARCH IS TO DEVELOP A HEART DISEASE PREDICTION SYSTEM.
* SPECIFIC OBJECTIVE: PROVIDES TO APPROACH CONCEALED PATTERNS IN THE DATA GIVEN TO AVOID HUMAN DEATHS ! 

In [ ]:
df_new.cp = df_new.cp.map({0:"asymptomatic", 1: "typical angina", 2:"atypical angina", 3:"non-anginal pain"})
df_new.sex = df_new.sex.map({0:"Female", 1:"Male"}) 
df_new.exang = df_new.exang.map({0:"exercise did not induce angina", 1:"exercise induced angina"})
df_new.slope = df_new.slope.map({1:"upsloping", 2:"flat", 3:"downsloping"})
df_new.thal = df_new.thal.map({1:"normal",2:"fixed defect", 3:"reversable defect"})
df_new = pd.get_dummies(df_new, drop_first = True)
df_new.head(10)

Define the dataset for training and testing.

* MOTIVATION : THE MOTIVATION OF THIS REPORT IS TO SAVE HUMAN BEINGS AND RESOURCE OF MEDICAL CENTERS AND IMPROVE ACCURANCY OF DIOGNIOS.
* AND TO MAKE AN AWARE OF PEOPLE FOR BEING REDICULLOS FOR HEART DIEASE OF UNWANTED FOOD.

In [ ]:
X = df_new.drop("target", 1).values
y = df_new["target"].astype("int").values
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.25, random_state=10)

To see which factors are significantly affect the heart disease, Xgboost library is a easy way to predict the result.
We should optimize the parameters first, so we assume different max depth of classification, learning rate and number of trees. 
Secondly, use crossvalid method to make sure the parameters is optimised.

In [ ]:
model = XGBClassifier()
param = dict(model_max_depth=[3,5,7], model_learning_rate=[0.001,0.01, 0.1], model_n_estimators=[100,500])

cv = GridSearchCV(model, param_grid=param, cv=10, iid = True)
cv

Let's see which condition is the best.

In [ ]:
cv.fit(X_train, y_train)
test_pred = cv.predict(X_test)
cv.best_estimator_

Great! We got the learning rate equals to 0.1, max depth equals to 3 and number of trees equals to 100. Use the best condition to implement the XGBoost model. 

In [ ]:
max_depth_of_model = cv.best_estimator_.max_depth
best_learning_rate = cv.best_estimator_.learning_rate
best_estimator = cv.best_estimator_.n_estimators
best_reg_lambda = cv.best_estimator_.reg_lambda

model = XGBClassifier(max_depth=max_depth_of_model, learning_rate=best_learning_rate, n_estimators= best_estimator,n_jobs=1,)
model.fit(X_train, y_train)
yhat = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]
accuracy_score(yhat,y_test)

The accuracy of model is 82.4%, it's not bad.
After that, let's see the importance of each features.

In [ ]:
importances = model.feature_importances_
importances

Sort the features on the basic of importances. 

*LITERATURE SURVEY : PREVELENCE OF HYPERTENSION
                   : DAIBETIES INSIPIDUS
                   : STRENGTH DURATION CURVE
                    : ADJUSTED PRESENT VALUE.
                    

In [ ]:
inducies = np.argsort(importances)[::-1]
inducies

Arrange the features as dictionary data structure.

In [ ]:
feature_dict = dict()
for idx in inducies:
    feature_dict[list(df_new.drop("target",1).columns)[idx]] = float(importances[idx])
feature_dict

In [ ]:
y_pos = np.arange(len(feature_dict.keys()))
plt.bar(y_pos, list(feature_dict.values()), align = "center",color = "lightgreen")
plt.xticks(y_pos, list(feature_dict.keys()), rotation = 90)
plt.xlabel("feature")
plt.ylabel("ratio")
plt.title("feature importances")
plt.show()

OK! Now we see "thal_normal" is not an important feature, that is, we can delete the column from dataset. 
On the other hand, reversable defect and ca are much higher than others.  
We talked about the influence of features later.

As we mentioned in introduction, we hope the recall accuracy is high. Before getting the score, we calculate four parts of confusion matrix. 

First one is true positive. It means that the sample is positive example, and the prediction is also positive.
Second one is false negative. This means that the sample is positive example, but the prediction is negative, and this is what we care about.
Third one is false positive. This one is relatived to precision of model. Most time we care about precision more, but this time the recall rate is more important.
Last but not least, true negative is that the predicition and sample are all negative. It's a bit the same as true positive.

In [ ]:
tp,fn,fp,tn = confusion_matrix(y_test, yhat, labels=[1,0]).ravel()
tp,tn,fp,fn

The quantity of false negative is 1. Seems the recall rate will be above 90%. Let's plot the confusion matrix.

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    return ""

In [ ]:
cmx = confusion_matrix(y_test, yhat, labels=[1,0])
print(plot_confusion_matrix(cmx, classes=['target=1','target=0'],normalize= False,  title='Confusion matrix'))

Once we know the detail of the confusion matrix, we can calculate some types of accuracy.
In this type of dataset, the recall rate is much important than precision rate, because no one want to misdiagnose the patient who has heart disease.

In [ ]:
precision_rate = tp / (tp + fp)
recall_rate = tp / (tp + fn)
print("The precision rate is: ", precision_rate)
print("The recall rate is: ", recall_rate)

The recall rate is about 97%, that is, still 3% of patient who has heart disease will be misdiagnosed. But now it's good enough.

**** PROBLEM DEFINATION IS BELOW ****

Next, we figure out the ROC curve performance.
The two main parameters of ROC curve is TPR and FPR. TPR is called true positive rate, and also called hit rate or sensitivity. The meaning of TPR is the ratio of positive sample being determined correctly. On the other hand, FPR is the ratio of negative sample being determined incorrectly. That is, the formula is defined (1 - Specificity). Let's see the auc.

In [ ]:
fpr, tpr, threshold =roc_curve(y_test, y_proba)
fig, ax = plt.subplots()
ax.plot(fpr,tpr)
ax.plot([0,1], [0,1], transform = ax.transAxes, ls="--", c="0.3")
plt.xlim(0.0, 1.0)
plt.ylim(0.0, 1.0)
plt.xlabel("FPR or 1-Specificity")
plt.ylabel("TPR or Sensitivity")
plt.rcParams["font.size"] = 10
plt.grid(True)
plt.show()

In [ ]:
auc(fpr, tpr)

If the AUC is in the range of 0.5~1, that means the model has value of prediction. The higher the score is, the better the model is. Moreover, the model will be a perfect classifier if the AUC equals to one. We got the 0.9 of the AUC score, not bad!

Next, let's see the weight of each features to know which one strongly affects the possibility.

In [ ]:
perm = PermutationImportance(model, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = df_new.drop("target", 1).columns.tolist())

In [ ]:
* EXPERIMENTAL SETUP

Obviously, "heart disease" and "reversable defect" are highly effective to heart disease, and male also has higher possibility to get heart disease. Maximum heart rate plays an important role in getting heart disease, the result surprised me because it is positive related to the heart disease. The other result surprising me is trestbps(resting blood pressure), its weight is not higher than we think.

The "age" is negative related to the heart disease. So let's take a look at the partial dependence plot to see what happen.

In [ ]:
CATERIZATION PROCESS

In [ ]:
clf = GradientBoostingClassifier(learning_rate = best_learning_rate, n_estimators = best_estimator,
                                 max_depth = max_depth_of_model).fit(X_train, y_train)

feature_dict = dict(enumerate(df_new.drop("target", 1).columns))
feature_dict

In [ ]:
my_plots = plot_partial_dependence(clf, features = [0], X = X_test, 
                                   feature_names = df_new.drop("target", 1).columns[:1], grid_resolution=5)     

plt.subplots_adjust(top=1.0, right=1.0)
my_plots

As the figure shown, the probability of getting heart disease is higher whether the age is higher, that's reasonable. So we must focus on heart disease because we are getting older.

Next, we see how ca(number of major vessels colored by flourosopy) affects the heart disease.

In [ ]:
my_plots = plot_partial_dependence(clf, features = [7], X = X_test, 
                                   feature_names = df_new.drop("target", 1).columns[:8], grid_resolution=5)     

plt.subplots_adjust(top=1.0, right=1.0)
my_plots

According to the medicine knowledge, if the major vessels colored by flourosopy get more, it means that the risk of heart disease will be lower. So, vessel flourosopy examination for everyone is a very important process to diagnose whether the people has heart disease or not.

In [ ]:
my_plots = plot_partial_dependence(clf, features = [5], X = X_test, 
                                   feature_names = df_new.drop("target", 1).columns[:6], grid_resolution=5)     

plt.subplots_adjust(top=1.0, right=1.0)
my_plots

There is a formula that can calculate a person's maximun heart rate: 220 - age

But, see the partial dependence results, the faster the rate is, the higher the risk is.
Does it mean that the young persons are in the high risk of heart disease? Not really!
If the max heart rate become faster year by year, it is possible that the heart become weaker. 

In [ ]:
my_plots = plot_partial_dependence(clf, features = [1], X = X_test, 
                                   feature_names = df_new.drop("target", 1).columns[:2], grid_resolution=5)     

plt.subplots_adjust(top=1.0, right=1.0)
my_plots

    ****    DETAILS  ****

Surprisingly! Although the resting blood pressure is still positive related to the risk of heart disease, a person who has lower resting blood pressure has higher possibility to get heart disease.
From this article, "low blood pressure that causes an inadequate flow of blood to the body's organs can cause strokes, heart attacks, and kidney failure. The most severe form is shock."
  * REFERENCE:　https://www.kaggle,com (heart disease categorization)

Besides, the risk becomes much lower once the resting blood pressure is above 190 mmHg, that's also a suprising outcome.

CGC CHEST X RAY is an indicated features of electrocardiogram. It depresses during exercising, and doctors always diagonose the heart situation when they are in the depression. 

In [ ]:
my_plots = plot_partial_dependence(clf, features = [6], X = X_test, 
                                   feature_names = df_new.drop("target", 1).columns[:7], grid_resolution=5)     

plt.subplots_adjust(top=1.0, right=1.0)
my_plots

When the st segment depresses too much, that's means that the heart is lack of oxygen. On the other hand, a person with asymptomatic ST-depression during exercise has high risk of sudden cardiac death. This result is impressive.

In [ ]:
fig = plt.figure()
target_feature = (6, 13)
pdp, axes = partial_dependence(clf, X_train, target_feature,  grid_resolution=50)
XX, YY = np.meshgrid(axes[0], axes[1])
Z = pdp[0].reshape(list(map(np.size, axes))).T
ax = Axes3D(fig)
surf = ax.plot_surface(XX, YY, Z, rstride=1, cstride=1, cmap=plt.cm.BuPu, edgecolor='k')
ax.set_xlabel(feature_dict[target_feature[0]])
ax.set_ylabel(feature_dict[target_feature[1]])
ax.set_zlabel('Partial dependence')

ax.view_init(elev=20, azim=72)
plt.colorbar(surf)
plt.subplots_adjust(top=0.9)
plt.show()

From this chart, a person with asymptomatic st-depression and flat slope is under high risk of getting heart disease.

Finally, let's plot the shap summary to know the data distribution of each features.

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test, feature_names = list(feature_dict.values()), plot_type = "bar", color = "lightblue")

In [ ]:
shap.summary_plot(shap_values, X_test, feature_names = list(feature_dict.values()))

Obviously, we find some features are seperated completely, like ca(number of major vessels colored by flourosopy), reversable defect, atypical angina, slope_upsloping, restecg, sex_male etc. It's easy to see the trend of each features. Most of these features are expected but sex_male and exercise induced angina. From our basic knowledge, if someone has angina while exerciseing, this person may be at risk. But the shap plot shows us the opposite results, that's interesting.

Also, we plot the shap force plot of lots of persons at once. We can easily to understandard what causes each person toward heart disease. As shown below, we choose all the male to see the distribution.

**  THE ABOVE DATA GIVES THE BELOW  RESULT OF HEART DISEASE CATEGORIZATION OF MALE AND FEMALE **

In [ ]:
def shap_force_plot_of_data(model, dataset):
    explainer = shap.TreeExplainer(model)
    shap_value_for_sample = explainer.shap_values(dataset)
    shap.initjs()
    drivein_force = shap.force_plot(explainer.expected_value, shap_value_for_sample, dataset)
    return drivein_force

person_is_monitored = pd.DataFrame(X_test, columns = list(feature_dict.values()))
shap_force_plot_of_data(model, person_is_monitored[person_is_monitored["sex_Male"]==1])


##      CONCLUSION  ** 

HEART  diseases is the top one killer for many years. I think that the reasons are our lack of knowledge about heart disease and the life habits. According to the model and features analysis, we know which features that we can do regular self-examinations. 

I think the most obvious symptom is chest pain. There are three types of chest pain, but only atypical angina is strongly related to the heart disease. No matter which type of chest pain you have, go to the  respictevely doctor near you!

In addition, everyone should always keep an eye on the resting blood pressure. The ideal resting blood pressure is lower than 120mmHg, but if your blood pressure is much lower than the 180mmHg, it means that you are under high risk of heart disease. Besides, the problem will not be only heart when the blood pressure is higher than 150mmHg.

Lots of electronic devices that can measure heart rate, so it's easier to monitor your own. Record max heart rate to make sure that your heart is still healthy. Once the rate rises year by year, there must be something wrong with you. 

SO FINALLLY  this was the conclusion for the report of heart disease of male and female , this really was a experience  to for the first time internship from nit calicut of this heart disease categorization using machine learning.

 DISCUSSIONS : STRESS TESTING ( SEE A CLINICAL WEBSITE)
 * CHEST X-RAY
 * CT ( COMPUTERIZED TOMOGRAPHY) SCAN
 * CONTINUOS ECG MONITIRIZING( SOMETIMES ALSO CALLED HOLTER MONITARING)
 

* REFERENCES : WWW.KAGGLE.COM(HEART DISEASE CATEGORIZATION) INTERNET , SOCAIL MEDIA, NEWSPAPER AND BASIC KNOWLEDGE .
    * THANK YOU RESPECTED SIR FOR GIVING ME THIS OPPRTUNIY .